# Recopilación de datos para la muestra

En esta sección se generará una muestra de datos sintéticos, según los parámetros obtenidos del Instituto Nacional de Estadística (INE) de España.

Los datos recogen estadísticas sobre la siguiente información por comunidad autónoma:

* Población
* Edades
* Género
* Actividad física
* Asistencia a eventos (cine, deportivos, en directo, culturales)
* Calidad del aire (días de alta contaminación, PM10)
* Criminalidad
* Nivel educativo
* Estado civil
* Horas trabajadas al mes
* Salario bruto anual
* Satisfacción sanitaria

## Configuración básica

Importación de librerías necesarias, y otras configuraciones para el correcto funcionamiento de los scripts

In [ ]:
import pandas as pd
import numpy as np
import random
import warnings

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Establecemos una semilla para reducir la aleatoriedad de los datos
np.random.seed(123)
random.seed(123)

data_basepath = '..\\data\\spain_original_data\\'

## Funciones

Funciones necesarias para la generacion de datos a partir de las estadísticas del INE.

In [ ]:
# Generador inicial del dataframe de muestra
def initial_dataframe_generator(values, weights, size):
  """
  Genera un DataFrame de pandas con una única columna de valores muestreados
  de una lista con probabilidades (pesos) dados.

  Args:
    values (list): La lista de valores de los que se muestreará.
    weights (list): La lista de pesos (probabilidades) correspondientes a cada valor.
                  Debe tener la misma longitud que 'valores'. Si la suma es mayor a 1
                  los valores serán normalizados.
    size (int): El número de filas en el DataFrame generado.

  Returns:
    pd.DataFrame: Un DataFrame de pandas con una columna llamada 'Valor_Aleatorio'
                  que contiene los valores muestreados según los pesos.
  """
  if len(values) != len(weights):
    raise ValueError("Las listas de valores y pesos deben tener la misma longitud.")

  # Asegurarse de que los pesos suman 1, si no, normalizarlos
  weights_num = sum(weights)
  probability = [p / weights_num for p in weights]

  # Generar los valores aleatorios usando np.random.choice
  random_values = np.random.choice(values, size=size, p=probability)

  # Crear el DataFrame
  df_generado = pd.DataFrame({'comunidad_autonoma': random_values})

  return df_generado

In [ ]:
# Función encargada de elegir un valor aleatorio de entre una lista, con una probabilidad dada
def generate_categorical_column(df, column_name, values_list, values_weights):
  """
  Incluye una columna en un DataFrame con valores aleatorios de una lista,
  siguiendo una distribución de pesos dada.

  Args:
    df (pd.DataFrame): El DataFrame al que se añadirá la columna.
    column_name (str): El nombre de la nueva columna.
    values_list (list): La lista de posibles valores para la nueva columna.
    values_weights (list): La lista de pesos (probabilidades) correspondientes a
                           cada valor en 'values_list'. Debe tener la misma
                           longitud que 'values_list'.

  Returns:
    pd.DataFrame: El DataFrame original con la nueva columna añadida.
                  Retorna el DataFrame original si hay un error en los pesos.
  """
  if len(values_list) != len(values_weights):
    print("Error: Las listas de valores y pesos deben tener la misma longitud.")
    return df

  # Normalizar los pesos para que sumen 1 y puedan usarse como probabilidades
  total_weights = sum(values_weights)
  if total_weights == 0:
      print("Error: La suma de los pesos es cero.")
      return df
  probabilities = [w / total_weights for w in values_weights]

  # Generar los valores aleatorios para la nueva columna
  # El tamaño debe ser igual al número de filas del DataFrame
  num_rows = len(df)
  random_values = random.choices(values_list, weights=probabilities, k=num_rows)

  # Añadir la nueva columna al DataFrame
  df[column_name] = random_values

  return df

In [ ]:
# Generación de columnas categóricas en función de los pesos por comunidad autónoma
def include_categorical_values(df_to_include, df_weights, weight_columns, new_column_name, value_column="comunidad_autonoma"):
    """
    Aplica la función generate_categorical_column a un DataFrame por cada comunidad
    autónoma, usando los pesos de otro DataFrame de valores.

    Args:
      df_to_include (pd.DataFrame): El DataFrame al que se le añadirá la nueva columna.
                                   Debe contener una columna 'comunidad_autonoma'.
      df_weights (pd.DataFrame): El DataFrame que contiene los valores y pesos
                                por comunidad autónoma. Debe contener la columna
                                especificada en 'value_column' y las columnas
                                especificadas en 'weight_columns'.
      weight_columns (list): Una lista de nombres de columnas en df_weights que
                             contienen los pesos para cada categoría. Estos nombres
                             serán los valores posibles para la nueva columna.
      new_column_name (str): El nombre de la nueva columna que se añadirá a df_to_include.
      value_column (str): El nombre de la columna en df_weights que identifica la comunidad
                          autónoma (ej. 'comunidad_autonoma').

    Returns:
      pd.DataFrame: El DataFrame original (df_to_include) con la nueva columna añadida.
                    Retorna el DataFrame original si hay errores.
    """
    if value_column not in df_weights.columns:
        print(f"Error: La columna '{value_column}' no se encuentra en df_weights.")
        return df_to_include

    if not all(col in df_weights.columns for col in weight_columns):
        missing_cols = [col for col in weight_columns if col not in df_weights.columns]
        print(f"Error: Faltan las siguientes columnas de peso en df_weights: {missing_cols}")
        return df_to_include

    if 'comunidad_autonoma' not in df_to_include.columns:
         print(f"Error: El DataFrame a incluir no tiene la columna 'comunidad_autonoma'.")
         return df_to_include

    # Crear una copia del DataFrame original para evitar SettingWithCopyWarning
    df_result = df_to_include.copy()

    # Lista para almacenar los DataFrames procesados por comunidad
    processed_dfs = []

    # Iterar sobre cada comunidad única en el DataFrame a incluir
    for comunidad in df_result['comunidad_autonoma'].unique():
        # Filtrar el DataFrame de valores para la comunidad actual
        comunidad_values = df_weights[df_weights[value_column] == comunidad]

        if comunidad_values.empty:
            print(f"Advertencia: No se encontraron datos de valores para la comunidad '{comunidad}'. Saltando.")
            # Añadir filas de la comunidad sin la nueva columna (se puede añadir un valor por defecto si es necesario)
            processed_dfs.append(df_result[df_result['comunidad_autonoma'] == comunidad])
            continue

        # Extraer los pesos para la comunidad actual
        # .iloc[0] para obtener la primera (y única) fila después del filtro
        weights_list = comunidad_values.loc[:, weight_columns].iloc[0].tolist()

        # Filtrar el DataFrame a incluir para la comunidad actual
        df_comunidad = df_result[df_result['comunidad_autonoma'] == comunidad]

        # Aplicar la función generate_categorical_column a la sub-DataFrame de la comunidad
        # Pasamos los nombres de las columnas de pesos como los valores posibles
        df_comunidad_processed = generate_categorical_column(
            df_comunidad,
            new_column_name,
            weight_columns,  # Los nombres de las columnas de peso son los valores
            weights_list     # Los valores dentro de esas columnas son los pesos
        )

        # Añadir el DataFrame procesado a la lista
        processed_dfs.append(df_comunidad_processed)

    # Concatenar todos los DataFrames procesados
    if processed_dfs:
        df_final = pd.concat(processed_dfs, ignore_index=True)
        return df_final
    else:
        print("No se procesó ninguna comunidad.")
        return df_to_include # Retorna el original si no se pudo procesar nada

In [ ]:
# Generación de datos numéricos en función de las medias por comunidad autónoma
def generate_numerical_column(df, df_means, new_col_name, mean_col_in_means_df, ca_col='comunidad_autonoma', std_dev=None):
    """
    Añade una columna con valores aleatorios con distribución normal a un DataFrame.

    Los valores se generan para cada fila basándose en la media correspondiente
    a la 'Comunidad_Autonoma' de esa fila, obtenida de otro DataFrame.

    Args:
        df (pd.DataFrame): DataFrame original con la columna 'ca_col'.
        df_means (pd.DataFrame): DataFrame que contiene las medias por 'ca_col'
                                  en la columna 'mean_col_in_means_df'.
        new_col_name (str): Nombre para la nueva columna a añadir en `df`.
        mean_col_in_means_df (str): Nombre de la columna en `df_means` que contiene
                                    los valores medios para cada comunidad.
        ca_col (str): Nombre de la columna que identifica la comunidad autónoma
                      en ambos DataFrames.
        std_dev (float): Desviación estándar para la distribución normal. Si no se
                         especifica, se usará el 25% la media global de los datos.

    Returns:
        pd.DataFrame: El DataFrame original con la nueva columna añadida.
    """
    # Crear un diccionario de mapeo de Comunidad Autónoma a su media
    mean_map = df_means.set_index(ca_col)[mean_col_in_means_df].to_dict()

    if std_dev == None:
        std_dev = df_means[mean_col_in_means_df].mean() / 4

    # Función para aplicar a cada fila y generar el valor aleatorio
    def generate_random_value(row):
        ca = row[ca_col]
        # Obtener la media para la CA de la fila, si no existe, usar una media por defecto
        mean = mean_map.get(ca, df_means[mean_col_in_means_df].mean()) # Usar media global si no se encuentra la CA

        # Generar un valor aleatorio con distribución normal
        random_value = np.random.normal(mean, std_dev)

        # Asegurar que el valor sea positivo
        return max(0, round(random_value, 4))

    # Aplicar la función a cada fila para crear la nueva columna
    df[new_col_name] = df.apply(generate_random_value, axis=1)

    return df

## Generación de la muestra

Es necesario crear una muestra inicial, proporcional a la población de cada Comunidad Autónoma, a la que posteriormente se irá añadiendo información de cada una de las estadísticas obtenidas.

In [ ]:
# GENERACIÓN DE MUESTRA DE POBLACIÓN
# Lista de posibles valores para comunidades autónomas
comunidades = ["Andalucía","Aragón","Asturias. Principado de","Balears. Illes",
               "Canarias","Cantabria","Castilla y León","Castilla - La Mancha",
               "Cataluña","Comunitat Valenciana","Extremadura","Galicia",
               "Madrid. Comunidad de","Murcia. Región de","Navarra. Comunidad Foral de","País Vasco",
               "Rioja. La","Ceuta","Melilla"]

# Pesos poblacionales ordenados correspondientes a cada comunidad
poblacion = [8472407,1326261,1011792,1173008,
             2172944,584507,2383139,2049562,
             7763362,5058138,1059501,2695645,
             6751251,1518486,661537,2213993,
             319796,83517,86261]

# Tamaño de la poblacion de muestra
DF_SIZE = 2000

# Generar el DataFrame
df_muestra = initial_dataframe_generator(comunidades, poblacion, DF_SIZE)

df_muestra

## Inclusión de columnas

Se irán incluyendo las columnas correspondientes a los datos procedentes del INE

### Edades de la población

Las edades de la población se establecen por rangos de 5 años, según `edades_por_comunidad.csv`

In [ ]:
# Leer el fichero con los datos de las edades
df_ages = pd.read_csv(f'{data_basepath}edades_por_comunidad.csv', delimiter=";")
df_ages

In [ ]:
# Columnas con los pesos
ages_weight_columns = [col for col in df_ages.columns if col != 'comunidad_autonoma']

# Obviamos los datos de la población menores de 10 años para este esudio
ages_weight_columns.remove("0-4_anos")
ages_weight_columns.remove("5-9_anos")
ages_weight_columns

In [ ]:
# Generamos la columna categórica "rango_edad" con los valores de "df_ages" como pesos
df_muestra = include_categorical_values(df_muestra, df_ages, ages_weight_columns, "rango_edad")
df_muestra

In [ ]:
# Establecemos un valor numérico para los rangos de edad
def set_random_age_in_range(rango_str):
  # Rango de edad X-Y_anos
  age_range = rango_str.replace('_anos', '').split('-')
  min_age = int(age_range[0])
  max_age = int(age_range[1])
  return random.randint(min_age, max_age)
  
# Aplicar la función a la columna 'rango_edad' para crear la nueva columna 'edad' con un dato numérico
df_muestra['edad'] = df_muestra['rango_edad'].apply(set_random_age_in_range)

In [ ]:
# Eliminamos la columna 'rango_edad' ya que no es necesaria
df_muestra.drop(columns=['rango_edad'], inplace=True)
df_muestra

### Género

In [ ]:
# Leer el fichero con los datos de sexo
df_gendre = pd.read_csv(f'{data_basepath}sexo_por_comunidad.csv', delimiter=";")

# Columnas con los pesos
gendre_weight_columns = [col for col in df_gendre.columns if col != 'comunidad_autonoma']
gendre_weight_columns

# Obviamos la columna total de "ambos sexos"
gendre_weight_columns.remove("ambos_sexos")

In [ ]:
# Generamos la columna categórica "genero" con los valores de "df_gendre" como pesos
df_muestra = include_categorical_values(df_muestra, df_gendre, gendre_weight_columns, "genero")

# sustituimos los valores de la columna "genero" por "hombre" y "mujer" en singular
df_muestra.replace({'genero': {'hombres': 'hombre', 'mujeres': 'mujer'}}, inplace=True)
df_muestra

### Actividad física de la población

La actividad física de la población se desglosa en `nivel_alto`, `nivel_moderado` y `nivel_bajo`.

In [ ]:
# Leer el fichero con los datos de la actividad fisica
df_activity = pd.read_csv(f'{data_basepath}actividadFisica_por_comunidad.csv', delimiter=";")

# Columnas con los pesos
activity_weight_columns = [col for col in df_activity.columns if col != 'comunidad_autonoma']
activity_weight_columns

In [ ]:
# Generamos la columna categórica "actividad_fisica" con los valores de "df_activity" como pesos
df_muestra = include_categorical_values(df_muestra, df_activity, activity_weight_columns, "actividad_fisica")
df_muestra

### Asistencia a eventos

La asistencia a eventos incluye varios tipos de eventos, por lo que se discriminará entre cada uno de los tipos para separar los datos

In [ ]:
# Leer el fichero con los datos de la asistencia a eventos
df_events = pd.read_csv(f'{data_basepath}asistenciaAEventos_por_comunidad.csv', delimiter=";")

# Columnas con los pesos
events_weight_columns = [col for col in df_events.columns if col != 'comunidad_autonoma']
events_weight_columns

In [ ]:
# Se separarán en varias columnas dependiendo del tipo de evento al que si/no/nopuede asistir (cine, directos, culturales y deportivos)
cine_columns = ['cine_si', 'cine_no_puede', 'cine_no']
directs_columns = ['directos_si', 'directos_no_puede', 'directos_no']
culturals_columns = ['culturales_si', 'culturales_no_puede', 'culturales_no']
sports_columns = ['deportivos_si', 'deportivos_no_puede', 'deportivos_no']

# Generamos las columnas categóricas "asistencia_---" con los valores de "df_events" como pesos
df_muestra = include_categorical_values(df_muestra, df_events, cine_columns, "asistencia_cine")
df_muestra = include_categorical_values(df_muestra, df_events, directs_columns, "asistencia_directos")
df_muestra = include_categorical_values(df_muestra, df_events, culturals_columns, "asistencia_cultural")
df_muestra = include_categorical_values(df_muestra, df_events, sports_columns, "asistencia_deporte")
df_muestra

In [ ]:
# Sustituimos los valores de cada columna cine, directos, culturales y deportivos por "si", "no_puede" y "no"
df_muestra.replace({'asistencia_cine': {'cine_si': 'si', 'cine_no_puede': 'no_puede', 'cine_no': 'no'}}, inplace=True)
df_muestra.replace({'asistencia_directos': {'directos_si': 'si', 'directos_no_puede': 'no_puede', 'directos_no': 'no'}}, inplace=True)
df_muestra.replace({'asistencia_cultural': {'culturales_si': 'si', 'culturales_no_puede': 'no_puede', 'culturales_no': 'no'}}, inplace=True)
df_muestra.replace({'asistencia_deporte': {'deportivos_si': 'si', 'deportivos_no_puede': 'no_puede', 'deportivos_no': 'no'}}, inplace=True)
df_muestra

### Calidad del aire

En los datos se muestra la calidad del aire como la media ponderada con la población del número de días al año en que se supera la concentración límite diaria de PM10 por comunidades autónomas (número de días)

In [ ]:
# Leer el fichero con los datos de la calidad de aire
df_air_quality = pd.read_csv(f'{data_basepath}calidadDelAire_por_comunidad.csv', delimiter=";")
df_air_quality.head()

In [ ]:
# Generación de los datos de calidad de aire
generate_numerical_column(df_muestra, df_air_quality, "dias_alta_contaminacion", "dias_alta_contaminacion")
df_muestra

### Homicidios y Criminalidad

In [ ]:
# Leer el fichero con los datos de homicidios y criminalidad
df_criminality = pd.read_csv(f'{data_basepath}criminalidad_por_comunidad.csv', delimiter=";")
df_criminality.head()

In [ ]:
# Generación de los datos de homicidios por cada 100.000 habitantes
generate_numerical_column(df_muestra, df_criminality, "homicidios_100mhabit", "homicidios_por_100mhabit")
df_muestra

In [ ]:
# Generación de los datos de criminalidad por cada 1.000 habitantes
generate_numerical_column(df_muestra, df_criminality, "criminalidad_1000habit", "criminalidad_por_1000habit")
df_muestra

### Nivel educativo

Los niveles educativos se reparten entre `analfabetos`, `estudios_primarios_incompletos`, `primaria`, `primero_secundaria`, `segundo_secundaria_general`, `segundo_secundaria_profesional_`, `educacion_superior`.

In [ ]:
# Leer el fichero con los datos de la nivel educativo
df_studies = pd.read_csv(f'{data_basepath}educacion_por_comunidad.csv', delimiter=";")

# Columnas con los pesos (obviando la columna 'total')
studies_weight_columns = [col for col in df_studies.columns if col != 'comunidad_autonoma' and col != 'total']
studies_weight_columns

In [ ]:
df_muestra = include_categorical_values(df_muestra, df_studies, studies_weight_columns, "estudios")
df_muestra

### Estado civil

In [ ]:
# Leer el fichero con los datos del estado civil
df_marital_status = pd.read_csv(f'{data_basepath}estadoCivil_por_comunidad.csv', delimiter=";")

# Columnas con los pesos (obviando la columna 'total')
marital_status_weight_columns = [col for col in df_marital_status.columns if col != 'comunidad_autonoma' and col != 'total']
marital_status_weight_columns

In [ ]:
df_muestra = include_categorical_values(df_muestra, df_marital_status, marital_status_weight_columns, "estado_civil")
df_muestra

### Horas trabajadas

Se incluye las horas trabajadas al mes por comunidad, calculando la media de los datos periódicos del año 2024

In [ ]:
# Leer el fichero con los datos de las horas trabajadas
df_work = pd.read_csv(f'{data_basepath}horasTrabajadasMes_por_comunidad.csv', delimiter=";")
df_work.head()

In [ ]:
# Cálculo de la media de los cuatro períodos
# df_work['media_2024'] = df_work.mean(axis=1)
df_work['media_2024'] = df_work[['2024T1', '2024T2', '2024T3', '2024T4']].mean(axis=1)

# Generación de los datos de salario
generate_numerical_column(df_muestra, df_work, "horasTrabajadas_mes", "media_2024")

df_muestra

### Salario anual

Se disponen de las medias de los salarios anuales medios de las comunidades autónomas

In [ ]:
# Leer el fichero con los datos del salario anual
df_salary = pd.read_csv(f'{data_basepath}salario_bruto_anual_por_comunidad.csv', delimiter=";")
df_salary.head()

In [ ]:
# Generación de los datos de salario
generate_numerical_column(df_muestra, df_salary, "salario_anual", "ambos_sexos")
df_muestra

### Satisfacción con el sistema sanitario

Se han recopilado datos del nivel de satisfacción entre varios servicios sanitarios:

* Hospitales
* Dentistas
* Especialistas
* Medicina General

Las valoraciones, ordenadas de mejor valorado a menor, varían entre:

> `Muy Satisfecho` > `Satisfecho` > `Neutral` > `Insatisfecho` > `Muy Insatisfecho`

In [ ]:
# Leer el fichero con los datos de la calidad sanitaria
df_health = pd.read_csv(f'{data_basepath}satisfaccionSanidad_por_comunidad.csv', delimiter=";")

# Columnas con los pesos
health_columns = [col for col in df_health.columns if col != 'comunidad_autonoma']
health_columns

In [ ]:
# Se separarán en varias columnas dependiendo del tipo de atención a valorar (hospital, dentista, especialista y medicina general)
hospitals_columns = ["hospitales_muy_satisfecho/a", "hospitales_satisfecho/a", "hospitales_neutral", "hospitales_insatisfecho/a", "hospitales_muy_insatisfecho/a"]
dentist_columns = ["dentistas_muy_satisfecho/a", "dentistas_satisfecho/a", "dentistas_neutral", "dentistas_insatisfecho/a", "dentistas_muy_insatisfecho/a"]
especialist_columns = ["especialistas_muy_satisfecho/a", "especialistas_satisfecho/a", "especialistas_neutral", "especialistas_insatisfecho/a", "especialistas_muy_insatisfecho/a"]
genMed_columns = ["medGeneral_muy_satisfecho/a", "medGeneral_satisfecho/a", "medGeneral_neutral", "medGeneral_insatisfecho/a", "medGeneral_muy_insatisfecho/a"]

# Generamos las columnas categóricas "asistencia_---" con los valores de "df_events" como pesos
df_muestra = include_categorical_values(df_muestra, df_health, hospitals_columns, "satisf_hospitales")
df_muestra = include_categorical_values(df_muestra, df_health, dentist_columns, "satisf_dentistas")
df_muestra = include_categorical_values(df_muestra, df_health, especialist_columns, "satisf_especialistas")
df_muestra = include_categorical_values(df_muestra, df_health, genMed_columns, "satisf_medGeneral")
df_muestra

## Guardado de fichero

In [ ]:
# prompt: guarda un dataframe en un archivo csv, en la ruta basepath, con el nombre "datos_tratados.csv"

# Definir la ruta completa del archivo CSV
filename = 'datos_tratados.csv'
file_path = f'{data_basepath}..\\spain_dataframes\\{filename}'

# Guardar el DataFrame en un archivo CSV
df_muestra.to_csv(file_path, index=False)

print(f"DataFrame guardado exitosamente en '{filename}'")